# Carga de Datos

## Librerías que se utilizarán

In [2]:
import os
import pandas as pd
import re
import datetime


### Con el siguiente bloque de código crearemos dataframes con los csv proporcionados

In [30]:
#contaminantes_catalogos
path_catalogos = r'/Users/sebastian/PycharmProjects/RegresionLinealSimple/data/contaminantes_catalogos/'
path_cdmx = '/Users/sebastian/PycharmProjects/RegresionLinealSimple/data/contaminantes_CDMX/'

files_catalogos = os.listdir(path_catalogos)
files_cdmx = os.listdir(path_cdmx)
#files_cdmx
files_catalogos


['cat_unidades.csv', 'cat_estacion.csv', 'cat_parametros.csv']

In [32]:
path_csv_cdmx = []
for file in files_cdmx:path_csv_cdmx.append(path_cdmx+file)

In [33]:
for i, archivo in enumerate(path_csv_cdmx):
    print(i)
    if i == 0:
        df = pd.read_csv(archivo, sep = ',', encoding='latin-1',  on_bad_lines='skip',skiprows=10,header=0)
    else:
        df = pd.concat([df , pd.read_csv(archivo, sep = ',', encoding='latin-1',  on_bad_lines='skip',skiprows=10,header=0)])


df.info()
df.head()

0
1
2
3
4
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9931032 entries, 0 to 2266535
Data columns (total 5 columns):
 #   Column        Dtype  
---  ------        -----  
 0   date          object 
 1   id_station    object 
 2   id_parameter  object 
 3   value         float64
 4   unit          int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 454.6+ MB


,date,id_station,id_parameter,value,unit
0,01/01/2022 01:00,ACO,CO,1.1,15
1,01/01/2022 01:00,ACO,NO,10.0,1
2,01/01/2022 01:00,ACO,NO2,39.0,1
3,01/01/2022 01:00,ACO,NOX,50.0,1
4,01/01/2022 01:00,ACO,O3,3.0,1


In [31]:
path_csv_cat = []
#path_csv_cat = [os.path.join(path_catalogos, n) for n in file_list_catalogos]
i =0
for file in files_catalogos:
    df = pd.read_csv(os.path.join(path_catalogos, file), skiprows=10, sep=',',  encoding='latin-1', header=1)
    path_csv_cat.append(df)
df = pd.read_csv(path_csv_cat[0], sep=',',  encoding='latin-1', header=0)
df.head()


TypeError: argument of type 'method' is not iterable

In [12]:
df = pd.read_csv(path_csv_cat[1], sep=',',  encoding='latin-1', header=1)
df.head()


,cve_estac,nom_estac,longitud,latitud,alt,obs_estac,id_station
0,ACO,Acolman,-98.912003,19.635501,2198.0,NaN,484150020109
1,AJU,Ajusco,-99.162611,19.154286,2942.0,NaN,484090120400
2,AJM,Ajusco Medio,-99.207744,19.272161,2548.0,NaN,484090120609
3,ARA,Aragón,-99.074549,19.470218,2200.0,Finalizó operación en 2010,484090050301
4,ATI,Atizapan,-99.254133,19.576963,2341.0,NaN,484150130101


In [6]:
df = pd.read_csv(path_csv_cat[2], sep=',',  encoding='latin-1', header=1)
for c in df.columns:
    print( c , df[c].map(type).unique().tolist()  , "\n"*1 )

id_parameter [<class 'int'>] 

cve_param [<class 'str'>] 

nom_param [<class 'str'>] 

unidades_param [<class 'str'>] 



In [10]:
df['dat_e'] = pd.to_datetime(  df['date'], format="%d/%m/%Y %H:%M", errors='coerce' )

df.head()

,date,id_station,id_parameter,value,unit,dat_e
0,01/01/2022 01:00,ACO,CO,1.1,15,2022-01-01 01:00:00
1,01/01/2022 01:00,ACO,NO,10.0,1,2022-01-01 01:00:00
2,01/01/2022 01:00,ACO,NO2,39.0,1,2022-01-01 01:00:00
3,01/01/2022 01:00,ACO,NOX,50.0,1,2022-01-01 01:00:00
4,01/01/2022 01:00,ACO,O3,3.0,1,2022-01-01 01:00:00


### Ingeniería de Variables


## Problemática : unidad muestral, matriz de predictoras, variable objetivo, generacion de variables, ventanas de tiempo, tad

## Creamos la um y nuestras anclas


In [11]:
um = ['hrs_indice', 'id_station', 'id_parameter']

In [11]:
df['date']=df['date'].astype(str)
df.date.dtypes

dtype('O')

In [12]:
df['hrs_indice'] = df['date'].astype(int)


ValueError: invalid literal for int() with base 10: '01/01/2022 01:00'

In [ ]:
catalogodehoras = df[['hrs']].drop_duplicates().reset_index(drop=True)
catalogodehoras['ancla'] = catalogodehoras.index -1


In [ ]:
'''Generación de anclas'''
vdesempenio = 1
vobs = 6
hrInicial = catalogodehoras['ancla'].min()
hrFinal = catalogodehoras['ancla'].max()

In [ ]:
ancla_Inicial = hrInicial + vobs
ancla_Final = hrFinal-vdesempenio
df = df.merge(catalogodehoras, on=['hrs_indice'], how='inner').reset_index(drop=True)


In [ ]:
df.info()